In [1]:
import os
import shutil
import pandas as pd


### ADULTS: Only move files from RFW full also in balanced adulst csv folder
**OBS: to file structure as child data**

In [7]:
adults_csv = pd.read_csv('../data/adults_balanced.csv')
len(adults_csv)

3306

In [20]:
def move_adults(image_dir, dst_root):
    image_file_names = adults_csv['img_org_name'].tolist()
        
    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir, d) for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of subfolders 
        folders2 = [os.path.join(folder, d) for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
        for folder2 in folders2:
        # Get the list of image files in the folder
            files = [f for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]

            # Check if any of the image file names from the CSV are in the folder
            for file in files:
                if file[:-4] in image_file_names:
                    # Move the image file to the output directory
                    
                    # Make folder with ethnicityname_imageid 
                    folder2_add = folder2.split("/")[-2:]
                    folder2_add_real = '_'.join(folder2_add)
                    dst_root = dst_root
                    dst = os.path.join(dst_root, folder2_add_real)
                    
                    # Check if the destination folder exists
                    if os.path.exists(dst):
                        print(f"Destination folder '{folder2_add_real}' already exists")
                    else:
                        # Create the destination folder if it doesn't exist
                        os.makedirs(dst, exist_ok=True)
                        #print(f"Created destination folder '{folder2_add_real}'")
        
                    src = os.path.join(folder2, file)
                    shutil.copy(src, dst)
                    
    print('Done moving files')

In [22]:
image_dir = '../data/raw_full/RFW/data'
dst_root = '../data/raw_full/adults'

move_adults(image_dir, dst_root)

Done moving files


### CHILDREN: Only move files from YLFW full and RFW full also in balanced children csv folder
**OBS: RFW to file structure as child data**

In [6]:
children_csv = pd.read_csv('../data/child_balanced.csv')
len(children_csv)

3306

In [39]:
def move_children(image_dir_rfw, image_dir_ylfw, dst_root):
    
    # First for RFW
    rfw_csv = children_csv[children_csv.img_org_name.notna()]
    image_file_names = rfw_csv['img_org_name'].tolist()
        
    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir_rfw, d) for d in os.listdir(image_dir_rfw) if os.path.isdir(os.path.join(image_dir_rfw, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of subfolders 
        folders2 = [os.path.join(folder, d) for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
        for folder2 in folders2:
        # Get the list of image files in the folder
            files = [f for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]

            # Check if any of the image file names from the CSV are in the folder
            for file in files:
                if file[:-4] in image_file_names:
                    # Move the image file to the output directory
                    
                    # Make folder with ethnicityname_imageid 
                    folder2_add = folder2.split("/")[-2:]
                    folder2_add_real = '_'.join(folder2_add)
                    
                    dst_root = dst_root
                    src = os.path.join(folder2, file)
                    dst = os.path.join(dst_root, folder2_add_real)
                    os.makedirs(dst,exist_ok=True)
                    shutil.copy(src, dst)
    print('Done moving RFW files')
    
    
    # THEN YLFW 
    ylfw_csv = children_csv[children_csv.img_org_name.isna()]
    ylfw_image_file_names = ylfw_csv['img_name'].tolist()
    
    # Get the list of folders in the image directory
    folders = [os.path.join(image_dir_ylfw, d) for d in os.listdir(image_dir_ylfw) if os.path.isdir(os.path.join(image_dir_ylfw, d))]
    # Iterate through each folder
    for folder in folders:
        # Get the list of image files in the folder
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        # Check if any of the image file names from the CSV are in the folder
        for file in files:
            if file[:-4] in ylfw_image_file_names:
                # Move the image file to same foldername in the output directory
                folder_dst = folder.split("/")[-1]
                dst_root = dst_root
                src = os.path.join(folder, file)
                dst = os.path.join(dst_root, folder_dst)
                os.makedirs(dst,exist_ok=True)
                shutil.copy(src, dst)
                
    print('Done moving YLFW files')
    

In [40]:
image_dir_rfw = "../data/raw_full/RFW/data"
image_dir_ylfw = "../data/raw_full/YLFW/data_aligned"
dst_root = "../data/raw_full/children"

move_children(image_dir_rfw, image_dir_ylfw, dst_root)

Done moving RFW files
Done moving YLFW files


### Check data folder sizes are equal

In [5]:
import os

def count_images_in_directory(directory):
    """
    Count the number of image files in a directory.
    """
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
    image_count = 0
    
    # Iterate through all files in the directory
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        
        # Check if the file is a regular file (not a directory)
        if os.path.isfile(file_path):
            # Check if the file has a recognized image extension
            if any(file_name.lower().endswith(ext) for ext in image_extensions):
                image_count += 1
                
    return image_count

def count_images_in_all_directories(root_directory):
    """
    Count the number of image files in all directories and subdirectories.
    """
    total_image_count = 0
    
    # Walk through all directories and subdirectories
    for dirpath, dirnames, filenames in os.walk(root_directory):
        # Count the number of image files in the current directory
        current_dir_image_count = count_images_in_directory(dirpath)
        
        # Add the count to the total image count
        total_image_count += current_dir_image_count
        
        # Print the directory path and the number of images
        print(f"Directory: {dirpath}, Images: {current_dir_image_count}")
    
    return total_image_count

# Example usage:
root_directory = '/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children'
total_images = count_images_in_all_directories(root_directory)
print(f"Total number of images: {total_images}")


Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children, Images: 0
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/African_754, Images: 1
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/African_566, Images: 1
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/Asian_m.0cj071, Images: 1
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/Indian_m.04mz6m8, Images: 2
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/Caucasian_m.0c3v046, Images: 1
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/African_330, Images: 2
Directory: /Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/raw_full/children/Caucasian_m.043qv9l, Images: 1
Directory: /Users/gabriellakierulff/Desktop/